In [38]:
import torch
import data_loader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import json
from collections import defaultdict

In [2]:
torch.cuda.is_available()

True

In [53]:
trainx, devx, testx, trainy, devy, testy = data_loader.load_all_classic_random_split(flatten=False)
# trainx, devx, testx, trainy, devy, testy = data_loader.load_all_subject_split(flatten=False)

Processing albert
Processing canon_12_5
Processing Chen_12_7
Processing daniel
Processing haobin_11_22
Processing isa_12_5
Processing janet
Processing joanne
Processing jq_12_6
Processing kelly_11_7
Processing kevin_11_7
Processing ruocheng
Processing russell_11_20_stand
Processing russell_11_7
Processing russell_random_12_7
Processing solomon
Processing wenzhou_12_5
Processing yiheng_11_30
Processing yiheng_12_5
Processing yongxu_11_30
Processing Zhaoye_12_7
Splitting out test set
Splitting out dev and train set


In [54]:
trainx, trainy = data_loader.augment_train_set(trainx, trainy, augment_prop=4, is_flattened=False)
trainx.shape, devx.shape, testx.shape, trainy.shape, devy.shape, testy.shape

Augmenting TRAIN set with proportion 4


((41600, 100, 3), (1040, 100, 3), (1041, 100, 3), (41600,), (1040,), (1041,))

In [69]:
BATCH_SIZE = 500

def get_dataloader(x, y, batch_size):
    dataset = [(x[i].T, y[i]) for i in range(y.shape[0])]
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader

trainloader = get_dataloader(trainx, trainy, BATCH_SIZE)
devloader = get_dataloader(devx, devy, BATCH_SIZE)
testloader = get_dataloader(testx, testy, BATCH_SIZE)

In [70]:
_, num_feature, num_channel = trainx.shape
num_feature, num_channel

(100, 3)

In [71]:
def acc_loss(data_loader, criterion):
    correct = 0
    total = 0
    total_loss = 0.0
    with torch.no_grad():
        for data in data_loader:
            x, y = data
            if torch.cuda.is_available():
                x = x.cuda()
                y = y.cuda()
                
            outputs = net(x.float())
            _, predicted = torch.max(outputs.data, 1)
            
            w = torch.sum((predicted - y) != 0).item()
            r = len(y) - w
            correct += r 
            total += len(y)
            
            total_loss += criterion(outputs, y.long()).item() * len(x)
    return correct / total, total_loss / total

In [72]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(
            in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1
        )  
        # 16 channel, num_feature
        self.pool1 = nn.MaxPool1d(kernel_size=3, stride=1, padding=1)
        # 16 channel, num_feature

        self.conv2 = nn.Conv1d(
            in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2
        )
        # 32 channel, num_feature
        self.pool2 = nn.MaxPool1d(kernel_size=5, stride=1, padding=2)
        # 32 channel, num_feature
        
        self.conv3 = nn.Conv1d(
            in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2
        )
        self.pool3 = nn.MaxPool1d(kernel_size=5, stride=1, padding=2)

        self.fc1 = nn.Linear(num_feature * 64, 3200)
        self.fc2 = nn.Linear(3200, 1600)
        self.fc3 = nn.Linear(1600, 500)
        self.out = nn.Linear(500, 26)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = x.view(-1, num_feature * 64)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.out(x)
        return x
net = Net()
if torch.cuda.is_available():
    net.cuda()
net

Net(
  (conv1): Conv1d(3, 16, kernel_size=(3,), stride=(1,), padding=(1,))
  (pool1): MaxPool1d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
  (conv2): Conv1d(16, 32, kernel_size=(5,), stride=(1,), padding=(2,))
  (pool2): MaxPool1d(kernel_size=5, stride=1, padding=2, dilation=1, ceil_mode=False)
  (conv3): Conv1d(32, 64, kernel_size=(5,), stride=(1,), padding=(2,))
  (pool3): MaxPool1d(kernel_size=5, stride=1, padding=2, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.25, inplace=False)
  (fc1): Linear(in_features=6400, out_features=3200, bias=True)
  (fc2): Linear(in_features=3200, out_features=1600, bias=True)
  (fc3): Linear(in_features=1600, out_features=500, bias=True)
  (out): Linear(in_features=500, out_features=26, bias=True)
)

In [73]:
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=0.00001, momentum=0.9)
# optimizer = optim.Adam(net.parameters(), lr=0.001)
optimizer = optim.AdamW(net.parameters(), weight_decay=0.1)

hist = defaultdict(list)
for epoch in range(20):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        if torch.cuda.is_available():
            inputs = inputs.cuda()
            labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs.float())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
        
    trainacc, trainloss = acc_loss(trainloader, criterion)
    devacc, devloss = acc_loss(devloader, criterion)
    hist['trainacc'].append(trainacc)
    hist['trainloss'].append(trainloss)
    hist['devacc'].append(devacc)
    hist['devloss'].append(devloss)
    
    print(f'Epoch {epoch} trainacc={trainacc} devacc={devacc}')
    print(f'        trainloss={trainloss} devloss={devloss}')

print('Finished Training')

Epoch 0 trainacc=0.37790865384615385 devacc=0.4009615384615385
        trainloss=2.019430611282587 devloss=2.0451032037918386
Epoch 1 trainacc=0.6906009615384615 devacc=0.6211538461538462
        trainloss=1.0157071811935077 devloss=1.3558958081098704
Epoch 2 trainacc=0.8387740384615384 devacc=0.6990384615384615
        trainloss=0.5124957016311013 devloss=1.2425738985721881
Epoch 3 trainacc=0.9067548076923077 devacc=0.7144230769230769
        trainloss=0.29004434388704026 devloss=1.3873514670592089
Epoch 4 trainacc=0.9458413461538462 devacc=0.7403846153846154
        trainloss=0.17698415426107553 devloss=1.5150411472870753
Epoch 5 trainacc=0.9634375 devacc=0.7326923076923076
        trainloss=0.12303681940270159 devloss=1.4985613754162421
Epoch 6 trainacc=0.971610576923077 devacc=0.7461538461538462
        trainloss=0.09888304095679465 devloss=1.6851233198092535
Epoch 7 trainacc=0.9806730769230769 devacc=0.7423076923076923
        trainloss=0.06708014573758611 devloss=1.67510600731923

In [60]:
testacc, testloss = acc_loss(testloader, nn.CrossEntropyLoss())
testacc, testloss
hist['testacc'] = testacc
hist['testloss'] = testloss

In [74]:
with open('cnn_hist_random.json', 'w') as f:
    json.dump(hist, f)